In [1]:
import numpy as np 
import pandas as pd

from PIL import Image

import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F

import albumentations
import torchvision
from torchvision import transforms

import timm
import tqdm

from src.losses import ArcMarginProduct
from src.utils.metrics import accuracy
from src.utils.data_loaders import ImagesDS
from matplotlib import pyplot as plt
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

import warnings
warnings.filterwarnings('ignore')

In [2]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [3]:
path_data = '/home/marchevskiy/Data/other'
device = 'cuda'
batch_size = 32

In [4]:
transforms_train = albumentations.Compose([
    albumentations.RandomCrop(height=334, width=334, p=1),
    albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.3),
    albumentations.RandomRotate90(p=0.3),
    albumentations.Flip(p=0.3),
    # albumentations.ElasticTransform(alpha=0.5, sigma=25, alpha_affine=25, p=0.3),
    # albumentations.RandomBrightness(limit=0.3, p=0.3),
    # albumentations.RandomContrast(limit=0.15, p=0.3),
    albumentations.Normalize(
     mean=[0.01639522, 0.06704897, 0.0366343 , 0.03555733, 0.02223296, 0.03011626],
     std=[0.01954379, 0.05231356, 0.01980171, 0.02657354, 0.01802711, 0.01548923])
])

transforms_test = albumentations.Compose([
    albumentations.RandomCrop(height=334, width=334, p=1),
    albumentations.Normalize(
     mean=[0.01639522, 0.06704897, 0.0366343 , 0.03555733, 0.02223296, 0.03011626],
     std=[0.01954379, 0.05231356, 0.01980171, 0.02657354, 0.01802711, 0.01548923])
])

In [5]:
ds = ImagesDS(pd.read_csv(path_data+'/train.csv'), path_data, transforms_train, mode='train')
ds_test = ImagesDS(pd.read_csv(path_data+'/test.csv'), path_data, transforms_test, mode='test')

loader = D.DataLoader(ds, batch_size=batch_size, shuffle=True, num_workers=4)
tloader = D.DataLoader(ds_test, batch_size=batch_size, shuffle=False, num_workers=4)

In [6]:
class DenseNet201(nn.Module):
    def __init__(self, num_classes=1108):
        super().__init__()
        preloaded = torchvision.models.densenet201(True)
        tail = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        tail.weight.data[:, :3, :, :] = preloaded..features.conv0.weight.data
        tail.weight.data[:, 3:, :, :] = preloaded..features.conv0.weight.data
        preloaded..features.conv0 = tail
        self.features = nn.Sequential(*list(preloaded.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.bn1 = nn.BatchNorm1d(1024 * 2)
        self.fc1 = nn.Linear(1024 * 2, 512 * 2)
        self.bn2 = nn.BatchNorm1d(512 * 2)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(512 * 2, 512)
        self.bn3 = nn.BatchNorm1d(512)
        del preloaded
        
    def forward(self, x):
        x = self.features(x)
        x = torch.cat((nn.AdaptiveAvgPool2d(1)(x), nn.AdaptiveMaxPool2d(1)(x)), dim=1)
        x = x.view(x.size(0), -1)
        x = self.bn1(x)
        x = F.dropout(x, p=0.25)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = F.dropout(x, p=0.5)

        x = x.view(x.size(0), -1)

        x = self.fc2(x)
        feature = self.bn3(x)

        return feature

SyntaxError: invalid syntax (<ipython-input-6-2f41201b9e1d>, line 6)

In [7]:
class ResNet50(nn.Module):
    def __init__(self, num_classes=1108):
        super().__init__()
        preloaded = torchvision.models.resnet50(True)
        tail = nn.Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        tail.weight.data[:, :3, :, :] = preloaded.conv1.weight.data
        tail.weight.data[:, 3:, :, :] = preloaded.conv1.weight.data
        preloaded.conv1 = tail
        self.features = nn.Sequential(*list(preloaded.children())[:-2])
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.bn1 = nn.BatchNorm1d(2048 * 2)
        self.fc1 = nn.Linear(2048 * 2, 512 * 2)
        self.bn2 = nn.BatchNorm1d(512 * 2)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(512 * 2, 512)
        self.bn3 = nn.BatchNorm1d(512)
        del preloaded
        
    def forward(self, x):
        x = self.features(x)
        x = torch.cat((nn.AdaptiveAvgPool2d(1)(x), nn.AdaptiveMaxPool2d(1)(x)), dim=1)
        x = x.view(x.size(0), -1)
        x = self.bn1(x)
        x = F.dropout(x, p=0.25)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = F.dropout(x, p=0.5)

        x = x.view(x.size(0), -1)

        x = self.fc2(x)
        feature = self.bn3(x)

        return feature

In [8]:
model = ResNet50()
# model = timm.models.efficientnet_b3(pretrained=False,
#                                     num_classes=1108,
#                                     in_chans=6)
# model.load_state_dict(torch.load('efficientnet_b3_v0.pth'))
# model.classifier = nn.Linear(1536, 512)
model.to(device);

In [8]:
import math

class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin
            cos(theta + m)
        """
    def __init__(self, in_features, out_features):
        super(ArcMarginProduct, self).__init__()
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        # nn.init.xavier_uniform_(self.weight)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)

    def forward(self, features):
        cosine = F.linear(F.normalize(features), F.normalize(self.weight.cuda()))
        return cosine
              
class ArcFaceLoss(nn.modules.Module):
    def __init__(self,s=30.0,m=0.5):
        super(ArcFaceLoss, self).__init__()
        self.classify_loss = nn.CrossEntropyLoss()
        self.s = s
        self.easy_margin = False
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, logits, labels, epoch=0):
        cosine = logits
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, labels.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        loss1 = self.classify_loss(output, labels)
        loss2 = self.classify_loss(cosine, labels)
        gamma=1
        loss=(loss1+gamma*loss2)/(1+gamma)
        return loss

In [9]:
criterion = ArcFaceLoss()
metric_fc = ArcMarginProduct(512, 1108)
optimizer = torch.optim.Adam([{'params': model.parameters()}, {'params': metric_fc.parameters()}])
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
metric_fc.to(device)

epochs = 1
tlen = len(loader)
for epoch in range(epochs):
    # scheduler.step()

    model.train()
    tloss = 0
    acc = np.zeros(1)
    for data in tqdm.tqdm_notebook(loader):
        data_input, label = data
        data_input = data_input.to(device)
        label = label.to(device).long()
        feature = model(data_input)
        output = metric_fc(feature)
        loss = criterion(output, label)
        tloss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = output.data.cpu().numpy()
        output = np.argmax(output, axis=1)
        label = label.data.cpu().numpy()

        acc += np.mean((output == label).astype(int))
    print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))

Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
  File "/usr/lib/python3.7/multiprocessing/queues.py", line 242, in _feed
    send_bytes(obj)
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.7/multiprocessing/connection.py", line 368, in 

KeyboardInterrupt: 

In [10]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

epochs = 1
tlen = len(loader)
for epoch in range(epochs):
    tloss = 0
    acc = np.zeros(1)
    model.train()
    for x, y in tqdm.tqdm_notebook(loader):
        x = x.to(device)
        optimizer.zero_grad()
        output = model(x)
        target = torch.zeros_like(output).to(device)
        target[np.arange(x.size(0)), y] = 1
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        tloss += loss.item() 
        acc += accuracy(output.cpu(), y)
        del loss, output, y, x, target
    print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))

RuntimeError: CUDA error: device-side assert triggered

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=30e-5)

epochs = 1
tlen = len(loader)
for epoch in range(epochs):
    # scheduler.step()

    model.train()
    tloss = 0
    acc = np.zeros(1)
    for data in tqdm.tqdm_notebook(loader):
        data_input, label = data
        data_input = data_input.to(device)
        label = label.to(device).long()
        feature = model(data_input)
        output = metric_fc(feature)
        loss = criterion(output, label)
        tloss += loss.item() 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        output = output.data.cpu().numpy()
        output = np.argmax(output, axis=1)
        label = label.data.cpu().numpy()

        acc += np.mean((output == label).astype(int))
    print('Epoch {} -> Train Loss: {:.4f}, ACC: {:.2f}%'.format(epoch+1, tloss/tlen, acc[0]/tlen))

RuntimeError: CUDA error: device-side assert triggered